In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! rm -rf /content/data_labeled/
! rm -rf /content/__MACOSX/
! ls -la

! unzip '/content/gdrive/MyDrive/data_labeled.zip'
! unzip '/content/gdrive/MyDrive/imgs.zip'

# Interactive Machine Learning - Exercise 03

In this exercise we will learn about cooperative machine learning.
Our goal is it to build a very basic cooperative machine learning user interface and use it to extend our Pokedex model from the last exercise.

The steps you are going to cover are as follows:
* Pretrain our Pokedex model with the original data
* Manually label a small bit of new data
* Train our model on the new data
* Use the model in a cooperative workflow to annotate the rest of the dataset

Please read each exercise carefully before you start coding! You will find a number in the comments before each step of coding you will do. Please refer to these numbers if you have any questions.

## 0. Import the libraries
As always we are providing a list useful packages in the import section below.
Keep in mind that you can import additional libraries at any time and that you do not need to use all the imports if you know another solution for a given task.

In [3]:
import ipywidgets as widgets
import os
import numpy as np
import glob
import random
from IPython.display import Image
from ipywidgets import interact_manual
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import Flatten, Dense, GlobalAveragePooling2D, Input
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from shutil import copyfile

import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## 1. Pretrain the model
In this part we are going to pretrain our model on the pokemon images you already know.
To this end we will use the same VGG16 model as last week with the following training procedure:

Preprocessing:
* Imagesize (224,224)
* Vgg16 standard preprocessing from the Keras framework

Datasplit:
* Use 90% of the data to train and 10% to valitdate your results

Training 1:
* Initialize the model with the imagenet weights
* Freeze all convolution layers
* Train the model using the following settings:
 * 5 Epochs
 * Adam Optimizer with default Parameters
 * categorical cross entropy loss
 * Batchsize 32

Training 2:
*  Unfreeze the last two convolutional Blocks
*  Continue training with the following settings:
 * 10 Epochs
 * Adam Optimizer with a learning rate of 0.0001
 * Batchsize of 32

A convolutional Block in the VGG16 architecture consists of 2 to 3 Conv Layers and on Pooling layer.
You can access a models layer directly via `model.layers`.
Read up on how to freeze layers [here](https://keras.io/guides/transfer_learning/), in case you did not use this technique in the last exercise.
Your model should achieve a validation accuracy of close to 100% .

In [4]:
# 1. Load data for pretraining and apply preprocessing
img_paths = [f'/content/imgs/{x}' for x in os.listdir('/content/imgs')]
print(len(img_paths), img_paths[0])

labels = []
for path in img_paths:
    label = path.split('/')[-1].split('_')[0]
    labels.append(label)

classes = list(set(labels))
print('distinct classes: ', classes)
n_classes = len(classes)

mapping = {k : classes.index(k) for k in classes}
print('mapping: ', mapping)
labels = [mapping[x] for x in labels]
print('encoded labels: ', labels[:3])

def one_hot(labels, depth):
    one_hot_vectors = []
    for label in labels:
        one_hot_vector = [0] * depth
        one_hot_vector[label] = 1
        one_hot_vectors.append(one_hot_vector)
    return one_hot_vectors

one_hot_labels = one_hot(labels, depth=8)
print('one hot encoded labels: ', one_hot_labels[:3])
one_hot_labels = np.array(one_hot_labels)

def read_image(fname):
    image = tf.io.read_file(fname)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(images=image, size=(224, 224))
    proto_tensor = tf.make_tensor_proto(image)
    image = tf.make_ndarray(proto_tensor)

    return image

x = []
for im_path in img_paths:
    img = read_image(im_path)
    x.append(img)

x = np.array(x)
print('x len:', len(x))
print('x instance shape: ', x[0].shape)

# 2. Split data into training and test partition
x_train, x_val, y_train, y_val = train_test_split(x, one_hot_labels,
                                                  test_size=0.10,
                                                  random_state=42)
print(len(x_train), len(x_val), len(y_train), len(y_val))

347 /content/imgs/VAPOREON_8.jpg
distinct classes:  ['CHARMANDER', 'EEVEE', 'FLAREON', 'PIKACHU', 'BULBASAUR', 'VAPOREON', 'SQUIRTLE', 'JOLTEON']
mapping:  {'CHARMANDER': 0, 'EEVEE': 1, 'FLAREON': 2, 'PIKACHU': 3, 'BULBASAUR': 4, 'VAPOREON': 5, 'SQUIRTLE': 6, 'JOLTEON': 7}
encoded labels:  [5, 7, 4]
one hot encoded labels:  [[0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 0, 0, 0]]
x len: 347
x instance shape:  (224, 224, 3)
312 35 312 35


In [5]:
# 3. Define network
base_model = tf. keras.applications.VGG16(weights="imagenet",
                                          include_top = False,
                                          input_shape=(224, 224, 3))
input_ = Input(shape=(224, 224, 3))
x_ = base_model(input_, training=False)
x_ = Flatten()(x_)
x_ = Dense(4096, activation='relu')(x_)
x_ = Dense(4096, activation='relu')(x_)
output_ = Dense(n_classes, activation='softmax')(x_)
pokexedx = Model(input_, output_)
pokexedx.summary()

# 4. Freeze weights and perform training step 1
pokexedx.trainable = False

opt = Adam()
pokexedx.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics='acc')
pokexedx.fit(x=x_train,
          y=y_train, 
          epochs=5,
          batch_size=32,
          validation_data=(x_val, y_val))

58892288/58889256 [==============================] - 1s 0us/step
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 4096)              102764544 
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 32776     
Total params: 134,293,320
Trainable params: 134,293,320

In [ ]:
# 5. Unfreeze weights and perform training step 2
base_model.summary()

print('\n'*3)
for layer in base_model.layers:
    if layer.name.startswith('block5_conv') or \
    layer.name.startswith('block4_conv'):
        print('unfreezing layer: ', layer.name)
        layer.trainable = True
print('\n'*3)

opt = Adam(learning_rate=0.0001)
pokexedx.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics='acc')
pokexedx.fit(x=x_train,
                    y=y_train, 
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

## 2. Pretrain the model
Now that we have our initial model we are going to extend it with some more pokemon.
[Here](https://megastore.uni-augsburg.de/get/OxpI3M_JyU/) you will find roughly 6000 images of the following Pokemon:
* Blastoise
* Charizard
* Charmeleon
* Ivysaur
* Venusaur
* Wartortle

Unfortunately images are not labeled yet. To speed things up a bit we are only going to label a small part of the data ourselves, and then build a model to help us doing the rest.
(Actually this will probably not be faster, but more fun anyway :) ).
In your project directory you will find a 'data_labled' folder, which we will use to store the labeled data.
This time we will use the folder structure to create our labels and train / validation partitions.
Inside the folder you will therefore find a 'train' and an 'val' folder, each of them containing subfolders for each class.

In the following step you should at first manually pick at least 5 examples per class and copy them from the 'data' folder to the train partition of the 'data_labeled' folder.
To then take full advantage of the current way the data is structured, we will use keras data generators in combination with the `flow_from_directory` to dynamically read the input data and feed it to our model.
You can find an example of such data generators [here](https://keras.io/api/preprocessing/image/#flowfromdirectory-method).

Specifically we are going to write a function `train_loop()` which creates two data generators (one for training and one for validation) and trains a model for the new Images on features extracted from our current Pokexedx model.
To this end you can simply rebuild the structure of the original model, but replace the number of output classes.
To load the weights you can then use the following code snippet:
`model.layers[-1]._name = 'new_output'`</br>
`model.load_weights(weight_path, by_name=True)`</br>

Freeze all layers but the dense layers, we will only need those and want to speed up the training process a bit.

In [7]:
weight_path = '/content/model_weights'
pokexedx.save_weights(weight_path,
                             overwrite=True,
                             save_format='h5',
)

In [ ]:
! wget https://megastore.uni-augsburg.de/get/OxpI3M_JyU/data.zip
! ls -la

In [ ]:
! unzip '/content/data.zip'

In [10]:
# 6. Copy at least 5 images per class from the data folder to the correct partition in the data_labeled folder
# made by hand?

# 7. Write a function train_loop()
classes = [x for x in os.listdir('/content/data_labeled/train') if x != '.DS_Store']
n_classes = len(classes)
print(n_classes, classes) 

def train_loop(n_classes, base_model, weights_path, verbose=0):
    # 8. Build model
    input_ = Input(shape=(224, 224, 3))
    x_ = base_model(input_, training=False)
    x_ = Flatten()(x_)
    x_ = Dense(4096, activation='relu')(x_)
    x_ = Dense(4096, activation='relu')(x_)
    output_ = Dense(n_classes, activation='softmax')(x_)
    model = Model(input_, output_)
    if verbose == 1:
        model.summary()

    # 9. Load weights
    model.load_weights(weights_path,
                       skip_mismatch=True,
                       by_name=True)

    opt = Adam(learning_rate=0.0001)
    model.compile(optimizer=opt,
                    loss='categorical_crossentropy',
                    metrics='acc')

    # 10. Build data generators
    train_datagen = ImageDataGenerator(rescale=1./255, vertical_flip=True)
    test_datagen = ImageDataGenerator(rescale=1./255)
    train_generator = train_datagen.flow_from_directory(
            '/content/data_labeled/train',
            target_size=(224, 224),
            batch_size=32,
            class_mode='categorical')
    validation_generator = test_datagen.flow_from_directory(
            '/content/data_labeled/val',
            target_size=(224, 224),
            batch_size=32,
            class_mode='categorical')

    # 11. Fit the model to the data for a few epochs
    model.fit(train_generator,
              epochs=10,
              verbose=verbose,
              validation_data=validation_generator)
    weights_path = '/content/model_new_weights'
    model.save_weights(weights_path,
                        overwrite=True,
                        save_format='h5',
    )
    model.save('/content/saved_model')

# 12. Call train loop
vgg_model = tf.keras.applications.VGG16(weights='imagenet',
                                        include_top=False,
                                        input_shape=(224, 224, 3))
vgg_model.trainable = False

train_loop(n_classes=n_classes,
            base_model=base_model,
            verbose=1,
            weights_path='/content/model_weights')

6 ['Wartortle', 'Ivysaur', 'Venusaur', 'Blastoise', 'Charmeleon', 'Charizard']
Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 4096)              102764544 
_________________________________________________________________
dense_4 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 24582     
Total params: 134,285,126
Trainable param

## 3. Interactive UI

In this part of the exercise we are going to put our pretrained model to good use by employing it in a cooperative workflow.
To this end we gonna build a minimal cooperative machine learning using interface in this python notebook.
Our user interface will consist of the following components:

* (optional) A progressbar to keep to motivation up
* A slider to set a high confidence threshold
* A slider to set the mid confidence threshold
* Some radio buttons to choose the label
* A button to save the annotation and label and show the next image
* A button to retrain our model
* A button to use our model to predict our dataset

The final our UI should look a like this:

![img](https://hcm-lab.de/cloud/index.php/s/ak3txGXepnt9NxS/preview)

The 'retrain' button should call the `train_loop()`  function from before to retrain the model on all labeled data.
The 'predict' button should create a list of predictions for all unlabeled images.
All predictions that are above the high confidence threshold, set by the respective slider, should be automatically accepted as correct label and copied to the respective folders in the training data folder.
Additionally you should implement a garbage label to delete unfitting images.
Potential reasons to consider an Image as garbage are if no Pokemon is visible, too many Pokemon are visible, non of the Pokemon we want to train are visible, the Imagefile is broken etc.
When you are pressing the 'next' button the current image should be copied to the right folder in the training dataset, depending on the current value of the radio button.
Afterwards the next image should be chosen from all predicted images, where the confidence is greater or equal than the value set by the mid_threshold slider.
The current value of the radiobutton should then be set to the prediction for this respective image.
Optionally you can also implement a progressbar to track your progress for you annotations.

You can use the ipywidgets library to create the UI.
You can find an IPython tutorial [here](https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6) and the api documentation [here](https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6).
Note, that Pycharm might not play well with the the widgets in all scenarios. It's best to view them in the browser by visting: http://localhost:8888 after you started your notebook.

In [12]:
# 13. Build UI
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import Image
from IPython.display import clear_output
from functools import partial
import shutil
import time
from tqdm import tqdm
import threading
np.set_printoptions(suppress=True)


class Button_Callback:
    def __init__(self):
        self.preds = None
        self.X = None
        self.cls = None

    def button_retr_callback(self, b):
        train_loop(n_classes=n_classes,
                    base_model=base_model,
                    weights_path='/content/model_new_weights')

    def radio_callback(self, b):
        self.cls = radio_buttons.value
        # print('='*50)
        # print('Assigned new class:', self.cls)
        if self.cls == 'Garbage':
            # print('Garbage class assigned, removing from unlabeled data...')
            os.remove(self.im_src)
            pass
        else:
            im_dst = f'/content/data_labeled/train/{self.cls}/{self.im_id}'
            # print('saving...', self.im_src, im_dst)
            shutil.copy(self.im_src, im_dst)
        print('next...')


    def button_next_callback(self, b):
        if self.mid_confs:
            self.im_src, pred_class = self.mid_confs[0]
            self.mid_confs.pop(0)

            self.im_id = self.im_src.split('/')[-1]
            # print('Predicted class: ', pred_class)
            file = open(self.im_src, "rb")
            image = file.read()
            im_widget = widgets.Image(value=image,
                                    format='jpeg',
                                    width=400,
                                    height=400)
            display(im_widget)
            file.close()
            print('You have 10 seconds to change the label!')
            time.sleep(10)
            im_widget.close()
        else:
            print('annotation done.')
        
    def button_pred_callback(self, b):
        # print('predicting...')
        path = '/content/saved_model'
        retr_model = tf.keras.models.load_model(path)
        # retr_model.summary()

        all_labeled = [x.split('/')[-1] for x in glob.glob('/content/data_labeled/train/*/*')]
        self.X = ['/content/data/' + x for x in os.listdir('/content/data/') if x not in all_labeled]

        imgs_to_pred = []
        for x in self.X[:3]:
            try:
                imgs_to_pred.append(read_image(x))
            except Exception as e:
                ...
                # print(f'Exception: {e}, image path: {x}')
        # print('creating array of images...')
        imgs_to_pred = np.array(imgs_to_pred)
        self.preds = retr_model.predict(imgs_to_pred)
        # print('predicted...', self.preds)
        self.pred_labels = [np.argmax(x) for x in self.preds]

        self.mid_confs = []

        for idx, pred in enumerate(self.preds):
            pred = pred[np.argmax(pred)]
            if pred >= high_conf_slider.value:
                # print(pred)
                im_src = self.X[idx]
                im_id = im_src.split('/')[-1]
                # print(im_src, im_id)
                # print(self.pred_labels[idx], classes)

                pred_class = classes[self.pred_labels[idx]]
                im_dst = f'/content/data_labeled/train/{pred_class}/{im_id}'
                # print(im_src, im_dst)
                shutil.copy(im_src, im_dst)
            elif high_conf_slider.value > pred >= mid_conf_slider.value:
                # print('mid: ', pred)
                im_src = self.X[idx]
                im_id = im_src.split('/')[-1]
                # print(im_src, im_id)
                # print(self.pred_labels[idx], classes)
                pred_class = classes[self.pred_labels[idx]]
                self.mid_confs.append([im_src, pred_class])
                # print(len(self.mid_confs), self.mid_confs)
        # print('done.')

callbacks = Button_Callback()

high_conf_slider = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.6, description='high conf')
mid_conf_slider = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.3, description='mid conf')

button_next = widgets.Button(description="Next")
button_next.on_click(callbacks.button_next_callback)

button_retr = widgets.Button(description="Retrain")
button_retr.on_click(callbacks.button_retr_callback)

button_pred = widgets.Button(description="Predict")
button_pred.on_click(callbacks.button_pred_callback)

classes_gargabe = classes + ['Garbage']
radio_buttons = widgets.RadioButtons(
            options=classes_gargabe,
            layout={'width': 'max-content'},
            description='Classes:'
        )

radio_buttons.observe(callbacks.radio_callback, names=['value'])

display(high_conf_slider)
display(mid_conf_slider)
display(radio_buttons)
display(button_retr)
display(button_pred)
display(button_next)

FloatSlider(value=0.6, description='high conf', max=1.0, step=0.01)

FloatSlider(value=0.3, description='mid conf', max=1.0, step=0.01)

RadioButtons(description='Classes:', layout=Layout(width='max-content'), options=('Wartortle', 'Ivysaur', 'Ven…

Button(description='Retrain', style=ButtonStyle())

Button(description='Predict', style=ButtonStyle())

Button(description='Next', style=ButtonStyle())

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00d\x00d\x00\x00\xff\xec\x00\x11Ducky\x00\x01\x00\…

You have 10 seconds to change the label!


![GUI](screenshot.png)

## 4. repeat(annotate, train, predict)
After you are done creating the UI, we are now going to label the whole dataset together with our model.
To this end use your model to predict and improve iteratively in the following manner:

Set the high confidence slider to a value greater or equal than 0.95 and the mid confidence slider to at least 0.8

Repeat 3 times:

* Call automatic prediction
* Check images that have been above the maximum confidence threshold manually by looking at the content of the respective folders. Make corrections if necessary.
* Annotate remaining images that have been over the mid confidence score
* Retrain you model

Do you notice any change in the amount of images you have to annotate each time?

Repeat till all data is annotated:

* Call automatic prediction
* Annotate remaining images that have been over the mid confidence score
* Retrain you model
* Adjust both confidence scores based on how much you trust your model

Describe your subjective impression of the annotation process. Did you have the feeling, that the cooperative workflow is helpful?

If base model accuracy is not sufficient to differentiate all possible classes and it overfits on only 1 class, then you have to set high confidence value to 1.0, because the model predicts only one class "Ivysaur" with the confidence of >= .99 for each image. Therefore you need to manually check all predictions.

From the other hand, if the model is performing well it is a very helpful tool to manually annotate images that model cannot assign class with desired confidence.
